In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

### Configure the model training policy


In [2]:
model_development.configure_training_policy()

### Load and preprocess data:

In [3]:
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv')
print([cell_images.shape, len(cell_labels), len(cell_types)])

# @interact(n = (0, cell_images.shape[0] - 1))
# def display_data(n = 0):
#     fig, axs = plt.subplots()
#     axs.imshow(cell_images[n], cmap = 'gray')
#     axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
#     return None

[(2624, 300, 300), 2624, 2624]


### Prepare data for model training

In [4]:
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


### Train classification model (first phase)


In [5]:
optimizer = model_development.create_optimizer('nadam')
inception_resnetv2 = \
    model_development.inception_resnetv2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_classification_model(\
    training_phase = 1, model = inception_resnetv2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'InceptionResNetv2', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.8831 - accuracy: 0.6396 - precision: 0.6509 - recall: 0.4847
Epoch 1: val_accuracy improved from -inf to 0.69552, saving model to ../models/weights\InceptionResNetv2_1.00.00_fs.hdf5
119/119 [==============================] - 41s 229ms/step - loss: 0.8831 - accuracy: 0.6396 - precision: 0.6509 - recall: 0.4847 - val_loss: 0.5674 - val_accuracy: 0.6955 - val_precision: 0.7356 - val_recall: 0.4476
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5544 - accuracy: 0.6887 - precision: 0.7470 - recall: 0.4994
Epoch 2: val_accuracy did not improve from 0.69552
119/119 [==============================] - 21s 177ms/step - loss: 0.5544 - accuracy: 0.6887 - precision: 0.7470 - recall: 0.4994 - val_loss: 0.6325 - val_accuracy: 0.6866 - val_precision: 0.6979 - val_recall: 0.4685
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5594 - accuracy: 0.7124 - precision: 0.7710 - recall: 0

### Train classification model (second phase)


In [6]:
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/InceptionResNetv2_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'InceptionResNetv2', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 844.7248 - accuracy: 0.5061 - precision: 0.4843 - recall: 0.9614
Epoch 1: val_accuracy improved from -inf to 0.46567, saving model to ../models/weights\InceptionResNetv2_1.00.00_ss.hdf5
119/119 [==============================] - 64s 418ms/step - loss: 844.7248 - accuracy: 0.5061 - precision: 0.4843 - recall: 0.9614 - val_loss: 334.5114 - val_accuracy: 0.4657 - val_precision: 0.4437 - val_recall: 0.9930
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 98.6329 - accuracy: 0.6723 - precision: 0.6435 - recall: 0.6617
Epoch 2: val_accuracy did not improve from 0.46567
119/119 [==============================] - 45s 381ms/step - loss: 98.6329 - accuracy: 0.6723 - precision: 0.6435 - recall: 0.6617 - val_loss: 397.7234 - val_accuracy: 0.4507 - val_precision: 0.4361 - val_recall: 0.9790
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 12.0149 - accuracy: 0.6902 - precision: 0.7597 